# 🏥 Medical Triage - Tool-as-Guide Pattern with Autonomous Agent

**⚠️ DISCLAIMER:** This is a simplified demonstration for educational purposes only. Real medical systems require clinical expertise, regulatory approval, and extensive validation. This is NOT production medical software.

---

## What This Demonstrates

This notebook shows the **Tool-as-Guide pattern** with an autonomous agent:

- **Guide (Workflow)**: Controls WHAT to do (protocol, steps, decisions)
- **Agent (Execution)**: Controls HOW to do it (tools, queries, processing)

### Key Insight

The agent is **autonomous** (calls tools, makes local decisions) but the **workflow is controlled** by the guide (can't skip steps, protocol is enforced).

This enables reliable AI for critical systems.


## Setup

Install dependencies (run once):


In [1]:
# Dependencies are installed via requirements.txt
# If you get import errors, make sure you:
#   1. Selected the "Medical Triage (Python 3.11)" kernel (Kernel → Change Kernel)
#   2. Ran: source .venv/bin/activate && uv pip install -r requirements.txt

import sys
print(f"✓ Using Python: {sys.executable}")
print(f"✓ Python version: {sys.version.split()[0]}")
print("\nMake sure you have:")
print("  1. Ollama installed (brew install ollama)")
print("  2. Gemma model downloaded (ollama pull gemma:2b)")
print("  3. Ollama running (ollama serve)")
print("  4. Selected the 'Medical Triage (Python 3.11)' kernel")


✓ Using Python: /Users/stelian.matei/Projects/cursor-extend/tool-as-guide-repo/examples/medical-triage/.venv/bin/python
✓ Python version: 3.11.6

Make sure you have:
  1. Ollama installed (brew install ollama)
  2. Gemma model downloaded (ollama pull gemma:2b)
  3. Ollama running (ollama serve)
  4. Selected the 'Medical Triage (Python 3.11)' kernel


## Imports


In [2]:
from triage_guide import MedicalTriageGuide
from fake_tools import MedicalDatabase, VitalsMonitor, SymptomClassifier, get_demo_scenario
import ollama
from rich.console import Console
from IPython.display import display, HTML, clear_output
import time
import sys

# Fix excessive line spacing and enable text wrapping
display(HTML("""
<style>
.jp-RenderedHTMLCommon pre {
    margin-top: 0px !important;
    margin-bottom: 0px !important;
    line-height: 1.3 !important;
    white-space: pre-wrap !important;
    overflow-x: auto !important;
}
</style>
"""))

console = Console(force_jupyter=True)
console.print("[bold green]✓ Imports successful[/bold green]")


✓ Imports successful

## Simple Autonomous Agent

This agent has four internal tools:
- **Guide** (workflow controller - protocol only)
- **Symptom Classifier** (domain knowledge - what's critical)
- **Database** (patient records)
- **Vitals Monitor** (vital signs)

The agent:
- Queries its guide for workflow decisions
- Uses classifier for medical domain knowledge
- Autonomously executes tasks
- Calls its tools as needed
- Reports back to its guide


In [3]:
class SimpleMedicalAgent:
    """Autonomous agent that executes tasks from the guide."""
    
    def __init__(self, llm_model="gemma:2b", use_llm=True):
        self.llm_model = llm_model
        self.use_llm = use_llm
        # Agent's tools
        self.classifier = SymptomClassifier()  # Domain knowledge
        self.db = MedicalDatabase()
        self.vitals = VitalsMonitor()
        self.guide = MedicalTriageGuide()  # Workflow controller
    
    def process_patient(self, patient_prompt, scenario="normal", delay_seconds=2):
        """
        Main entry point: Process a patient through the triage workflow.
        
        Args:
            patient_prompt: What the patient says
            scenario: Scenario type for demo purposes
            delay_seconds: Pause between steps for readability
        """
        
        console.print(f"[bold]👤 Patient:[/bold] \"{patient_prompt}\"")
        console.print()
        
        # Agent asks its guide: "What should I do?"
        console.print("[bold blue]🤖 Agent: Asking guide for instructions...[/bold blue]")
        result = self.guide.start_triage()
        session_id = result['session_id']
        
        console.print(f"[bold]📋 Guide:[/bold] {result['instructions_for_agent']}")
        console.print(f"   [dim]Protocol: {result['protocol']}[/dim]")
        console.print()
        
        time.sleep(delay_seconds)
        
        # Agent executes the task using its tools
        agent_report = self._execute_task(
            task=result['task'],
            instructions=result['instructions_for_agent'],
            patient_input=patient_prompt,
            scenario=scenario
        )
        console.print()
        
        time.sleep(delay_seconds)
        
        # Agent reports back to guide
        console.print("[bold blue]🤖 Agent: Reporting findings to guide...[/bold blue]")
        console.print(f"   [dim]Symptoms: {agent_report['symptoms_detected']}[/dim]")
        console.print(f"   [dim]Severity: {agent_report.get('classification', {}).get('severity', 'unknown')}[/dim]")
        console.print()
        
        result = self.guide.continue_triage(session_id, agent_report)
        
        console.print("[bold]📋 Guide: Evaluating based on protocol...[/bold]")
        
        # Check if we need to save record first (emergency case)
        if result['status'] == 'emergency_save_required':
            console.print(f"[bold red]📋 Guide: EMERGENCY PROTOCOL ACTIVATED[/bold red]")
            console.print(f"   [bold]Decision:[/bold] {result['decision']}")
            console.print(f"   [bold]Triage Level:[/bold] {result['triage_level']}")
            console.print(f"   [bold]Red Flags:[/bold] {', '.join(result['red_flags_detected'])}")
            console.print()
            console.print(f"[bold]📋 Guide:[/bold] {result['instructions_for_agent']}")
            console.print()
            
            time.sleep(delay_seconds)
            
            # Agent saves the record
            save_report = self._execute_task(
                task=result['task'],
                instructions=result['instructions_for_agent'],
                triage_data=result['triage_data']
            )
            console.print()
            
            time.sleep(delay_seconds)
            
            # Agent reports save completion back to guide
            console.print("[bold blue]🤖 Agent: Record saved, reporting to guide...[/bold blue]")
            result = self.guide.continue_triage(session_id, save_report)
            console.print("[bold]📋 Guide:[/bold] Emergency protocol complete.")
            console.print()
            console.print("[bold]Message to Patient:[/bold]")
            console.print(result['message'])
        elif result['status'] == 'emergency':
            console.print(f"[bold red]📋 Guide: EMERGENCY PROTOCOL ACTIVATED[/bold red]")
            console.print(f"   [bold]Decision:[/bold] {result['decision']}")
            console.print(f"   [bold]Triage Level:[/bold] {result['triage_level']}")
            console.print(f"   [bold]Red Flags:[/bold] {', '.join(result['red_flags_detected'])}")
            console.print()
            console.print("[bold]Message to Patient:[/bold]")
            console.print(result['message'])
        else:
            console.print(f"[bold]📋 Guide:[/bold] {result.get('instructions_for_agent', 'Continue with protocol')}")
            console.print(f"   [dim]Next task: {result.get('task')}[/dim]")
        
        console.print()
        time.sleep(delay_seconds)
        
        console.print("[bold]📊 Protocol Compliance Audit[/bold]")
        console.print()
        
        for step in result.get('audit_trail', []):
            step_name = step['step'].replace('_', ' ').title()
            timestamp = step['timestamp'].split('T')[1].split('.')[0]  # Just show time
            console.print(f"   ✓ {step_name} - {timestamp}")
        
        console.print()
        console.print("[bold green]✅ All required protocol steps completed[/bold green]")
        console.print("[bold green]✅ Emergency escalation protocol followed[/bold green]")
        console.print("[bold green]✅ System is auditable and compliant[/bold green]")
        
        # Don't return result to avoid displaying JSON in notebook output
    
    def _execute_task(self, task, instructions, patient_input=None, scenario="normal", triage_data=None):
        """Agent autonomously executes a task from the guide."""
        console.print(f"[bold blue]🤖 Agent: Executing task '{task}'...[/bold blue]")
        
        if task == "screen_red_flags":
            return self._screen_red_flags(patient_input)
        elif task == "gather_chief_complaint":
            return {"chief_complaint": patient_input}
        elif task == "check_medical_history":
            return self._check_medical_history(patient_input)
        elif task == "get_vital_signs":
            return self._get_vital_signs(scenario)
        elif task == "assess_severity":
            return {"assessment": "completed"}
        elif task == "save_triage_record":
            return self._save_triage_record(triage_data)
        else:
            return {"error": f"Unknown task: {task}"}
    
    def _screen_red_flags(self, patient_input):
        """Agent's implementation of red flag screening using its tools"""
        # Step 1: Get medical history
        console.print("   [dim]→ Agent: Calling medical_db.check_conditions()[/dim]")
        history = self.db.check_conditions(patient_input)
        console.print(f"   [dim]→ Agent: Found history - high_risk={history.get('high_risk')}[/dim]")
        
        # Step 2: Use classifier to assess symptom severity
        console.print("   [dim]→ Agent: Calling symptom_classifier.classify_symptoms()[/dim]")
        classification = self.classifier.classify_symptoms(patient_input, history)
        console.print(f"   [dim]→ Agent: Severity={classification['severity']}, Critical={len(classification['critical_symptoms'])}[/dim]")
        console.print("   [green]✓ Agent: Task completed[/green]")
        
        return {
            "symptoms_detected": classification['critical_symptoms'] + classification['moderate_symptoms'],
            "patient_statement": patient_input,
            "medical_history": history,
            "classification": classification
        }
    
    def _check_medical_history(self, patient_input):
        """Check patient medical history"""
        console.print("   [dim]→ Agent: Querying medical records[/dim]")
        history = self.db.check_conditions(patient_input)
        medications = self.db.get_medications()
        allergies = self.db.check_allergies()
        console.print(f"   [dim]→ Agent: Found {len(medications)} medications[/dim]")
        console.print("   [green]✓ Agent: Task completed[/green]")
        
        return {
            "medical_history": history,
            "medications": medications,
            "allergies": allergies,
            "high_risk_conditions": history.get("chronic_conditions", [])
        }
    
    def _get_vital_signs(self, scenario):
        """Get and assess vital signs"""
        console.print("   [dim]→ Agent: Calling vitals_monitor.get_vitals()[/dim]")
        vitals = self.vitals.get_vitals(scenario=scenario)
        critical = self.vitals.assess_vitals(vitals)
        console.print(f"   [dim]→ Agent: Vitals obtained[/dim]")
        if critical:
            console.print(f"   [bold red]⚠ Agent: Critical values: {critical}[/bold red]")
        console.print("   [green]✓ Agent: Task completed[/green]")
        
        return {"vitals": vitals, "critical_values": critical}
    
    def _save_triage_record(self, triage_data):
        """Save triage record to patient file"""
        console.print("   [dim]→ Agent: Calling medical_db.save_triage_record()[/dim]")
        patient_id = triage_data.get("patient_id", "P001")
        result = self.db.save_triage_record(patient_id, triage_data)
        console.print(f"   [dim]→ Agent: {result['message']} (ID: {result['record_id']})[/dim]")
        console.print("   [green]✓ Agent: Task completed[/green]")
        
        return result

console.print("[bold green]✓ Agent class defined[/bold green]")


✓ Agent class defined

## Initialize System


In [4]:
# Initialize agent (which contains all its tools internally)
agent = SimpleMedicalAgent()

console.print("\n" + "="*60)
console.print("[bold]🏥 MEDICAL TRIAGE SYSTEM INITIALIZED[/bold]")
console.print("="*60)
console.print("\n[bold]Agent's Tools:[/bold]")
console.print("  ✓ Workflow Guide (protocol enforcement - no domain knowledge)")
console.print("  ✓ Symptom Classifier (domain knowledge - what's critical)")
console.print("  ✓ Medical Database (patient records)")
console.print("  ✓ Vitals Monitor (vital signs)")
console.print("\n[bold]Architecture:[/bold]")
console.print("  • Guide: WHAT to do (workflow steps)")
console.print("  • Classifier: Domain expertise (medical knowledge)")
console.print("  • Agent: HOW to do it (autonomous execution)")
console.print("\nReady for triage.\n")


============================================================

🏥 MEDICAL TRIAGE SYSTEM INITIALIZED

============================================================

Agent's Tools:

✓ Workflow Guide (protocol enforcement - no domain knowledge)

...

Architecture:

• Guide: WHAT to do (workflow steps)

• Classifier: Domain expertise (medical knowledge)

• Agent: HOW to do it (autonomous execution)

Ready for triage.

---

## 🚨 Complete Demo: Emergency Case

Run the cell below to see the entire triage workflow in action.

**Note:** The output cell will scroll automatically in Jupyter if needed. Click the left margin of the output to collapse/expand. There are 2-second pauses between steps for readability.

**What to watch for:**
1. **Agent asks Guide** for instructions (workflow controlled)
2. **Guide responds** with protocol steps (no domain knowledge)
3. **Agent executes** using its tools (classifier, db, vitals)
4. **Agent reports** findings back to Guide
5. **Guide evaluates** and makes protocol decision
6. **Guide instructs** agent to save triage record
7. **Audit trail** records every step


In [6]:
# Patient presents with symptoms
patient_prompt = "I have severe chest pain that started 30 minutes ago"

# Agent processes the patient
agent.process_patient(patient_prompt)


👤 Patient: "I have severe chest pain that started 30 minutes ago"

🤖 Agent: Asking guide for instructions...

📋 Guide: CRITICAL: Before anything else, screen for emergency symptoms. This is mandatory protocol.

Protocol: Emergency Department Triage Protocol - Red Flag Screening (Mandatory)

🤖 Agent: Executing task 'screen_red_flags'...

...

✓ Emergency Escalation Activated - 17:53:52

✓ Triage Record Saved - 17:53:56

✅ All required protocol steps completed

✅ Emergency escalation protocol followed

✅ System is auditable and compliant

---

## Key Takeaways

### What Just Happened:

1. **Guide Controlled Workflow**
   - Mandated red flag screening (can't be skipped)
   - Detected emergency condition
   - Activated escalation protocol
   - Logged every step

2. **Agent Had Autonomy**
   - Decided which tools to call
   - Processed patient input
   - Queried databases
   - Compiled reports

3. **Protocol Was Enforced**
   - Consistent behavior
   - Can't skip critical steps
   - Auditable trail
   - Deterministic decisions

### Why This Matters:

Traditional approaches fail:
- **Prompts**: "Please check for emergencies" → might skip it
- **Pure agents**: Decide their own workflow → unpredictable
- **Workflow engines**: Control everything → no agent autonomy

**Tool-as-Guide**: Agent autonomy + workflow reliability = Safe AI for critical systems

---

## Try Different Scenarios

Restart and try different patient inputs:
- `"I have a mild headache"` → Normal triage flow
- `"I feel dizzy and confused"` → Different assessment
- `"Severe bleeding"` → Immediate escalation

**Notice:** Same workflow every time, but agent adapts to content
